In [1]:
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException,ElementNotVisibleException
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Connect to the web driver

driver = webdriver.Chrome(r"C:\Users\safiy\Downloads\chromedriver_win32 (2)\chromedriver.exe")

In [3]:
def cars24_search(search_term,location):
    """Generate a URL for search term with location"""
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001',
          'Jaipur':'302001' , 'Chandigarh':'160017' , 'Lucknow':'226001', 'Nagpur':'440001', 'Surat':'395003', 
          'Ludhiana':'141001' , 'Faridabad':'121002'}
    
    search_term = search_term.replace(' ','%20')
    template = f'https://www.cars24.com/buy-used-car?sort=P&search={search_term}&storeCityId=8597&pinId={LOC[location]}'
    
    return template

In [4]:
def cars24_scrapping():
    """ 
    Function to scrap datas from cars 24.
    
    """
      
    print("SCRAPPING THE REQUIRED DETAILS")
    driver = webdriver.Chrome(r"C:\Users\safiy\Downloads\chromedriver_win32 (2)\chromedriver.exe")
    driver.maximize_window()
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001',
          'Jaipur':'302001' , 'Chandigarh':'160017' , 'Lucknow':'226001', 'Nagpur':'440001', 'Surat':'395003' ,
          'Ludhiana':'141001' , 'Faridabad':'121002'}
    
    location = [keys for keys in LOC.keys()]
    
    CARS= ['SUV', 'Sedan', 'Hatchback', 'Luxury Sedan', 'Luxury SUV' , 'Maruti' , 'Toyota','MAHINDRA', 'Tata']
    
    URL = [cars24_search(j,i) for i in location for j in CARS]
    
    LOCATION = []
    MNF_YEAR = []
    BRAND = []
    MODEL = []
    VARIANT = []
    driven_KM = []
    NOOF_owners = []
    fuel_type = []
    PRIZE = []


    for i in URL:
        driver.get(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class' : '_1l4fi'})

        description = []
        description1 = []
        description2 = []
        price = []
    
        for i in results:
            description.append(i.find('h2', {'class': "_3FpCg"}).text)
            description1.append(i.find('p', {'class': "cvakB"}).text)
            description2.append(i.find('ul', {'class': "bVR0c"},'li').text)
            price.append(i.find('div', {'class': "_7udZZ"}).text)
            #LOCATION.append(driver.find_element(By.XPATH,"//p[@class='_1OR7d']/label").text)
    
    
        year = [i.split(' ',2)[0] for i in description]
        brand = [i.split(' ',2)[1] for i in description]
        D1= [i.split(' ',2)[-1] for i in description]

        for i in price:
            PRIZE.append(i)

        for i in year:
            MNF_YEAR.append(i)
        for i in brand:
            BRAND.append(i)
    
        D2 = [i.rsplit(' ',1)[0] for i in description1]
        variant = [i.split(' ') [-1]for i in description1]
        for i in variant:
            VARIANT.append(i)
    
        D3 = [D1[i]+' '+D2[i] for i in range(0,len(D1))]
    
        for i in D3:
            MODEL.append(i)
    
        dummy1 = [i.split('km')[-1]for i in description2]
        kms = [i.split('km')[0] for i in description2]
        kms = [i+'km' for i in kms]

        owners = [i.split('Owner')[0] for i in dummy1]
        owners = [i+'Owner' for i in owners]
        fuel = [i.split('Owner')[-1] for i in dummy1]

        for i in kms:
            driven_KM.append(i)

        for i in owners:
            NOOF_owners.append(i)

        for i in fuel:
            fuel_type.append(i)

    driver.close()
        
    used_cars = pd.DataFrame({
                              "MNF_YEAR" : MNF_YEAR,
                              "BRAND" : BRAND,
                              "MODEL" : MODEL,
                              "VARIANT" : VARIANT,
                              "DRIVEN_KM" : driven_KM,
                              "FUELTYPE" : fuel_type,
                              "NOOF_OWNERS" : NOOF_owners,
                              "PRICE" : PRIZE
                             })
    
    used_cars.to_csv("used_cars.csv",index=False) #Creating CSV
    print("SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'")
    
    return used_cars

In [55]:
user_cars = cars24_scrapping()


SCRAPPING THE REQUIRED DETAILS
SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'


In [56]:
user_cars


,MNF_YEAR,BRAND,MODEL,VARIANT,DRIVEN_KM,FUELTYPE,NOOF_OWNERS,PRICE
0,2021,Mahindra,XUV 3OO 1.2 W6 MT,,"31,273 km",PetrolDL-3C,1st Owner,"₹10,95,000"
1,2020,Mahindra,XUV 3OO 1.2 W6 MT,,"30,557 km",PetrolDL-10,1st Owner,"₹9,19,000"
2,2019,Mahindra,XUV 3OO 1.5 W6 MT,Manual,"41,356 km",DieselUP-14,1st Owner,"₹8,30,000"
3,2022,Mahindra,XUV 3OO 1.2 W6 AT,,"22,158 km",PetrolDL-1C,1st Owner,"₹10,36,000"
4,2020,Mahindra,KUV 100 NXT K8 6 STR,Manual,"26,959 km",PetrolDL-8C,1st Owner,"₹6,94,000"
...,...,...,...,...,...,...,...,...
6609,2018,Tata,Tiago XZA 1.2 REVOTRON,Automatic,"52,957 km",PetrolDL-3C,1st Owner,"₹5,30,000"
6610,2019,Tata,Tiago XZA 1.2 REVOTRON,Automatic,"39,318 km",PetrolHR-26,1st Owner,"₹5,73,000"
6611,2020,Tata,ALTROZ XZ 1.2,Manual,"16,787 km",PetrolHR-26,1st Owner,"₹7,84,000"
6612,2020,Tata,ALTROZ XZ 1.2,Manual,"23,053 km",PetrolHR-26,1st Owner,"₹7,38,000"
